# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect("data.sqlite")

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [10]:
# Your code here
pd.read_sql(
    """
    select firstName, lastName
    from employees
    left join offices
        using (officeCode)
    where lower(city) = 'boston';
    """, conn
)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [16]:
# Your code here
pd.read_sql(
    """
    select officeCode, city, count(*) as num_employees
    from offices
    left join employees
        using (officeCode)
    group by
        officeCode, city;
    """, conn
)

,officeCode,city,num_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2
7,27,Boston,1


## Write 3 questions of your own and answer them

In [18]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

'\nHow many customers are there per office?\n'

In [26]:
"""
What is the most popular product?
"""

# Your code here
pd.read_sql(
    """
    select 
        productVendor, productCode, productName, 
        sum(quantityOrdered) as unitsOrdered,
        sum(priceEach * quantityOrdered) as totalRevenue
    from orderDetails
    left join products
        using (productCode)
    group by
        productVendor, productCode, productName
    order by
        totalRevenue desc, unitsOrdered desc
    limit 5;
    """, conn
)

,productVendor,productCode,productName,unitsOrdered,totalRevenue
0,Unimax Art Galleries,S18_3232,1992 Ferrari 360 Spider red,1808,276839.98
1,Second Gear Diecast,S12_1108,2001 Ferrari Enzo,1019,190755.86
2,Classic Metal Creations,S10_1949,1952 Alpine Renault 1300,961,190017.96
3,Red Start Diecast,S10_4698,2003 Harley-Davidson Eagle Drag Bike,985,170686.00
4,Autoart Studio Design,S12_1099,1968 Ford Mustang,933,161531.48


In [30]:
"""
Which product has the most expensive MSRP?
"""

# Your code here
pd.read_sql(
    """
    select distinct productName, MSRP
    from products
    order by
        MSRP desc
    limit 5;
    """, conn
)

,productName,MSRP
0,1952 Alpine Renault 1300,214.30
1,2001 Ferrari Enzo,207.80
2,1968 Ford Mustang,194.57
3,2003 Harley-Davidson Eagle Drag Bike,193.66
4,1969 Ford Falcon,173.02


In [32]:
"""
Which employee has made company the most revenue?
"""

# Your code here
pd.read_sql(
    """
    select 
        employeeNumber, firstName, lastName,
        sum(amount) as revenue
    from employees as e
    join customers as c
        on e.employeeNumber = c.salesRepEmployeeNumber
    join payments as p
        using (customerNumber)
    group by
        employeeNumber, firstName, lastName
    order by
        revenue desc
    limit 5;
    """, conn
)

,employeeNumber,firstName,lastName,revenue
0,1370,Gerard,Hernandez,1112003.81
1,1165,Leslie,Jennings,989906.55
2,1401,Pamela,Castillo,750201.87
3,1501,Larry,Bott,686653.25
4,1504,Barry,Jones,637672.65


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [38]:
# Your code here
pd.read_sql(
    """
    select
        firstName, lastName, productName
    from employees as e
    join customers as c
        on e.employeeNumber = c.salesRepEmployeeNumber
    join orders as o
        using (customerNumber)
    join orderdetails
        using (orderNumber)
    join products
        using (productCode)
    order by
        firstName, lastName;
    """, conn
)

,firstName,lastName,productName
0,Andy,Fixter,1996 Moto Guzzi 1100i
1,Andy,Fixter,2003 Harley-Davidson Eagle Drag Bike
2,Andy,Fixter,P-51-D Mustang
3,Andy,Fixter,1936 Harley Davidson El Knucklehead
4,Andy,Fixter,1997 BMW R 1100 S
...,...,...,...
2991,Steve,Patterson,2002 Suzuki XREO
2992,Steve,Patterson,1928 Ford Phaeton Deluxe
2993,Steve,Patterson,1930 Buick Marquette Phaeton
2994,Steve,Patterson,American Airlines: B767-300


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [48]:
# Your code here
pd.read_sql(
    """
    select
        firstName, lastName, count(productCode) as productCount
    from employees as e
    join customers as c
        on e.employeeNumber = c.salesRepEmployeeNumber
    join orders as o
        using (customerNumber)
    join orderdetails
        using (orderNumber) 
    group by
        firstName, lastName
    order by
        productCount desc
    limit 5;
    """, conn
)

,firstName,lastName,productCount
0,Gerard,Hernandez,396
1,Leslie,Jennings,331
2,Pamela,Castillo,272
3,Larry,Bott,236
4,Barry,Jones,220


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [46]:
# Your code here
pd.read_sql(
    """
    select
        firstName, lastName, count(distinct productCode) as productCount
    from employees as e
    join customers as c
        on e.employeeNumber = c.salesRepEmployeeNumber
    join orders as o
        using (customerNumber)
    join orderdetails
        using (orderNumber) 
    group by
        firstName, lastName
    having
        productCount > 200
    order by
        productCount desc
    limit 5;
    """, conn
)

,firstName,lastName,productCount


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!